# Ryan Sikhrangkur

# Data Wrangling Report

<br>

## Table of Contents
- [Introduction](#Introduction)
- [Gathering Data](#Gathering-Data)
- [Assessing Data](#Assessing-Data)
- [Cleaning Data](#Cleaning-Data)
- [Reassessment & Conclusion](#Reassessment-and-Conclusion)

<br>

## Introduction <a class="anchor" id="Introduction"></a>

This report will summarize the process in which I wrangled data from a tweet archive of the user *@dog_rates*, otherwise known as *WeRateDogs*. With over four-million followers, *WeRateDogs* rates users' dogs and includes humorous commentary talking about them.

I was able to wrangle this data using Python and several of its libraries, including Pandas, NumPy, and the Twitter API Tweepy.

My wrangling report will be divided into the three major steps of wrangling: **gathering**, **assessing**, and **cleaning**. I will also include any reassessments performed, and difficulties faced in the wrangling process.

<br>

## Gathering Data <a class="anchor" id="Gathering-Data"></a>

I collected the data from three sources:
- A `.csv` file, named `twitter-archive-enhanced.csv`.
- A `.tsv` file created through web scraping.
- A `.json` file created through the service of a Twitter API.

The `twitter-archive-enhanced.csv` file I was able to read into a dataframe table using Pandas and named it `df_archive`.

My second source of data was from a [webpage](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv), which I was able to scrape using the requests library and save to an `image_predictions.tsv` file. From that file, I read the data into a dataframe table using Pandas and named it `df_predictions`.

The final source of information I gathered through Tweepy. Tweepy is a Twitter API which I applied to use for the purpose of this project, and using the list of tweet IDs present in `df_archive` saved the favorite and retweet counts to a `.json` file. I was then able to use the json library to read the file into a dictionary, and use Pandas to create a dataframe table named `df_likes_rts` from the dictionary.

<br>

## Assessing Data <a class="anchor" id="Assessing-Data"></a>

I assessed the data both visually and programmatically to uncover any issues there may be in the three tables I created from what I had gathered. For the scope of this project I followed two requirements:
1. The tweets must all be original, i.e. they must not be retweets.
2. There should be no tweets made beyond the date of August 1st, 2017.

I began my assessment with simple operations such as printing the tables for a broader view and calling the tables' info to see their structure (columns and their data types, null entry values).

Afterwards I investigated specific pieces of data - checking if columns existed in multiple tables, the unique values of several columns and how many of each there were, and looking for data that would be invalid (e.g. a `rating_denominator` of zero in `df_archive`).

### Issues

My assessment of the data provided a look into several key issues which needed to be resolved to better prepare the data for any exploratory data analysis. The issues were defined into two categories.

Recall that an issue with **quality** refers to problems with the data's content. If data is invalid, corrupted, or duplicated, then the data is low quality.

Recall that an issue with **tidiness** refers to the structure of the data. Each variable forms a column, each observation forms a row, and each observational unit forms a table.

The issues I've discovered are as follows.

### Quality Issues
#### df_archive
1. There are 181 non-original tweets which need to be excluded.
2. The `in_reply_to_status_id`, `in_reply_to_user_id`, `source` and `expanded_urls` columns do not provide useful information. 
3. The `name` column has entries which aren't names, and null values are listed as the `str` value "None".
4. The `tweet_id` column stores `int` type values instead of `str`.
5. The `timestamp` field stores `str` type values instead of `date`.

#### df_predictions
1. Several column names are difficult to understand.
2. The values entered in the `p1`, `p2` and `p3` columns have inconsistent capitalization and punctuation practices.
3. The `tweet_id` column stores `int` type values instead of `str`.

#### df_likes_rts
1. The `tweet_id` column stores `int` type values instead of `str`.

### Tidiness Issues
1. 181 tweets are missing from `df_predictions`, and two weets are missing from `df_likes_rts`.
2. The `df_archive` has one data type (dog stage) stored in four columns (`doggo`, `floofer`, `pupper`, `puppo`).
3. The `df_likes_rts` table provides little additional information and can be joined to the `df_archive` dataset.

<br>

## Cleaning Data <a class="anchor" id="Cleaning-Data"></a>

Before any cleaning I created copies of the three tables to preserve the originals, then I cleaned the issues identified in the assessment with the following priority in mind:
1. Tidiness issues pertaining to missing data.
2. Other tidiness issues.
3. Quality issues, in order of established tables.

### Tidiness Issues

**1. Missing tweets in `df_predictions` and `df_likes_rts`.**

I saw there was missing data when the tables had different numbers of observations.
- `df_archive` had 2356.
- `df_likes_rts` had 2354.
- `df_predictions` had 2075.

To correct this I had set all tables to include observations which only contained `tweet_id` values which were present in all three.

**2. One data type in four columns in `df_archive`**

Some tweets mention a dog stage, which is stored into an appropriate column. These stages are `doggo`, `floofer`, `pupper` and `puppo`. For the columns that aren't mentioned, or if no dog stage is mentioned, these values are saved as "none".

I saw this as an opportunity to clean this structure by creating one column named `dog_stage`, and the other four columns are saved as the observation values in this column. Values of "none" are convered to `np.nan` to better reflect the existence of null values. The `doggo`, `floofer`, `pupper`, and `puppo` columns are dropped from the table.

**3. The `df_likes_rts` table doesn't provide substantial information.**

Because `df_likes_rts` provided only two unique column variables, I saw it best to merge the `df_likes_rts` with the `df_archive` table joined on the `tweet_id` column.

### Quality Issues

#### df_archive

**1. Non-original tweets must be excluded.**

It was possible to identify non-original tweets by locating observations with a non-null value in the `retweeted_status_id`, `retweeted_status_user_id`, and/or `retweeted_status_timestamp` variable columns.

Knowing this, I had excluded any observations where those variables were not null from `df_archive`. These columns were then dropped from the table.

**2. Several columns are not useful.**

There are some columns present which didn't provide information I felt would be useful towards exploratory data analysis, and was inflating the table size too much.

In the previous issue I addressed retweets, but no mention of replies which are noted by the `in_reply_to_status_id` and `in_reply_to_user_id` columns.

Also, a tweet can be identified and all its contents viewed through the `tweet_id` alone, which rendered the `source` and `expanded_urls` columns redundant.

It is for these reasons I dropped the four columns from the table.

**3. The `name` column contains invalid values and "none" instead of null.**

The `name` column in `df_archive` stores values in tweets where an algorithm uses speech patterns to detect, or more accurately "predict", the names of these dogs where the speech patterns are present. There are several values which were stored using this algorithm which aren't names, but instead common words (an, the, etc.). These values were removed, and along with the value "none" replaced with `np.nan` to reflect an accurate count of tweets with no value for `name`.

**4. The `tweet_id` column stores `int` variables instead of `str`.**

Tweet IDs are numerical, yet the `int` value type is susceptible to unwanted mathematical processes, such as a tweet ID leading with 0s but must remain a consistent length. I changed the data type of the `tweet_id` variable to `str` to prevent manipulation of these values.

**5. The `timestamp` column stores `str` variables instead of `date`.**

Likewise to the last issue, I converted the `timestamp` variable to the `date` data type to prevent errors which can be made if the column continues to store `str` values (misspelling of the month, inconsistent time and date formats, etc.). 

#### df_predictions

**1. Several column names are difficult to understand.**

Columns such as `p_1` and `p1_conf` may not make sense to data analysts who are newer to the project. I renamed these columns for legibility purposes, from `p_n` to `prediction_n` and `pn_conf` to `pn_confidence`. I also renamed `img_num` to `img_count` to better reflect the quantity of images found in each observation.

**2. Values in the `prediction_1`, `prediction_2` and `prediction_3` columns have inconsistent grammar practices.**

To remain consistent and more legible, I had all observations adjusted to follow the same grammar practices.

**3. The `tweet_id` column stores `int` variables instead of `str`.**

Just as in the `df_archive` table, I change the data type of the `tweet_id` column from `int` to `str` to better preserve the values and prevent unwanted manipulation.

<br>

## Reassessment & Conclusion <a class="anchor" id="Reassessment-and-Conclusion"></a>

Once the above issues were resolved, I addressed two new issues.

**1. There are less tweets in `df_archive` than in `df_predictions`.**


This issue is likely the result of removing retweets from the `df_archive` table, but neglecting to check for those retweets in `df_predictions`. I resolved this by repeating the process of excluding observations whose `tweet_id` values were not present in both tables.

**2. Some observations in `df_archive` have multiple dog stages as a value in `dog_stage`.**

It's acceptable for a dog to have multiple values in `dog_stage`, however in these instances they were combined into one word (e.g., "doggofloofer" instead of "doggo, floofer"). I located these values, and corrected them to reflect the multiple `dog_stage` values.<br>

Once I completed all assessment and cleaning of the data, I merged the two tables into one `df_master` table and saved it to a `.csv` file for future analysis.

While I consider my cleaning objectives complete, there is much more which can be done to make the data more suitable. For example, using regular expressions (Regex) it's possible that more speech patterns can be considered and give us more dog names where it's assumed there are none.

This concludes my wrangling report.